# Curso Modelado Numérico de la Atmósfera

Alumno: *Gerardo Rivera Tello*

---

## Tarea 2

Dada la siguiente malla de datos de temperatura, pronosticar la evolución del campo de la variable para una hora. Aplicar el método de Euler para las derivadas zonal y meridional.

### Datos
* Velocidad del viento = CONST = 7m/s
* Dirección del viento = CONST = 140°
* Resolución del dominio dx = dy = 18km
* Paso del tiempo ¿?

$$
TEMP = \begin{bmatrix}
21.2 & 22.8 & 23.5 & 24.8 & 26.5 \\
22.7 & 24.4 & 25.2 & 26.5 & 28.4 \\
24.3 & 26.1 & 27.0 & 28.4 & 30.4 \\
26.0 & 27.9 & 28.9 & 30.4 & 32.5 \\
27.8 & 29.9 & 30.9 & 32.5 & 34.8 \\
29.7 & 32.0 & 33.0 & 34.8 & 37.2 \\
31.8 & 34.2 & 35.3 & 37.2 & 39.8
\end{bmatrix}$$

---

## Solución

La ecuación que estamos tratando de resolver es la siguiente:

$$
\frac{\partial T }{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} = 0
$$

En donde el paso de tiempo estara determinado por

$$
\frac{\lvert u\rvert*\Delta t}{\Delta x} + \frac{\lvert v\rvert*\Delta t}{\Delta y} \leq 0.06
\\
$$

Que resulta en

$$
\Delta t \leq \frac{0.06}{\left(\frac{\lvert u\rvert}{\Delta x} + \frac{\lvert v\rvert}{\Delta y}\right)}
$$

In [ ]:
# Manejo de datos
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# Plots y graficos interactivos
import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn
from panel.interact import fixed, interact

pn.extension()
hv.extension("bokeh")

In [ ]:
# Declaración de datos
dx = dy = 18
magw = 7
angw = 140
temp_arr = np.array(
    [
        [21.2, 22.8, 23.5, 24.8, 26.5],
        [22.7, 24.4, 25.2, 26.5, 28.4],
        [24.3, 26.1, 27.0, 28.4, 30.4],
        [26.0, 27.9, 28.9, 30.4, 32.5],
        [27.8, 29.9, 30.9, 32.5, 34.8],
        [29.7, 32.0, 33.0, 34.8, 37.2],
        [31.8, 34.2, 35.3, 37.2, 39.8],
    ]
)

In [ ]:
# Calculo de las componentes u,v
u = magw * np.cos(np.deg2rad(140))
v = magw * np.sin(np.deg2rad(140))

print(f"{u = }\n{v = }")

Con los valores obtenidos para u y v, calculamos el valor límite que debe tener $\Delta t$

In [ ]:
dt_approx = 0.06 / ((np.abs(u) / (dx * 1000)) + (np.abs(v) / (dy * 1000)))
print(f"{dt_approx = }")

Entonces tomamos

$$
\Delta t \approx 100s
$$

Construimos una grilla que cumpla con $\Delta x$ y $\Delta y$ (no es necesario realizar esto, pero es util para tener coordenadas definidas dentro de xarray)

In [ ]:
lon = np.arange(-90, -90 + temp_arr.shape[1] * dx, dx)
lat = np.arange(-50, -50 + temp_arr.shape[0] * dy, dy)
print(f"{lon = }\n{lat = }")

Construimos nuestra matriz de datos con `lat` y `lon` como coordenadas

In [ ]:
temp = xr.DataArray(temp_arr, coords=[("lat", lat[::-1]), ("lon", lon)])

# Metadata adicional para variables/coordenadas que sigue la convención CF
temp.lon.attrs["long_name"] = "Longitude"
temp.lon.attrs["units"] = "degrees_east"
temp.lat.attrs["long_name"] = "Latitude"
temp.lat.attrs["units"] = "degrees_north"
temp.attrs["long_name"] = "Temperature"
temp.attrs["units"] = "degrees_C"
temp.name = "temperature"

temp

In [ ]:
# Arreglos para grafico de quivers
u_arr = np.full_like(temp_arr, u)
v_arr = np.full_like(temp_arr, v)

Se puede explorar nuestra malla de datos iniciales

In [ ]:
fig, ax = plt.subplots(dpi=100)
temp.plot(ax=ax, vmin=20, vmax=40)
ax.quiver(lon, lat, u_arr, v_arr)
ax.set_title("En grilla")

In [ ]:
levels = np.arange(20, 40.5, 0.5)

fix, ax = plt.subplots(dpi=100)
temp.plot.contourf(ax=ax, vmin=20, vmax=40, levels=levels)
CS = temp.plot.contour(ax=ax, vmin=20, vmax=40, colors="k", levels=levels[::4])
ax.quiver(lon + dx / 2, lat + dy / 2, u_arr, v_arr)
ax.set_ylim(lat.min(), lat.max())
ax.set_xlim(lon.min(), lon.max())
ax.clabel(CS, colors="k", fmt="%i")
ax.set_title("En contornos")

---

La discretización de la ecuacion a resolver se hara con un esquema centrado en el espacio y adelantado en el tiempo

$$
\left(\frac{T_{i,j}^{n+1} - T_{i,j}^{n}}{\Delta t}\right) + u \left(\frac{T_{i+1,j}^{n} - T_{i-1,j}^{n}}{2\Delta x} \right) + v \left( \frac{T_{i,j+1}^{n} - T_{i,j-1}^{n}}{2\Delta y} \right) = 0
$$

De donde

$$
T_{i,j}^{n+1} = \Delta t \ T_{i,j}^{n} - \frac{u \Delta t}{2 \Delta x} \left( T_{i+1,j}^{n} - T_{i-1,j}^{n} \right) - \frac{v \Delta t}{2 \Delta y} \left( T_{i,j+1}^{n} - T_{i,j-1}^{n} \right)
$$

Implementamos la discretizacion obtenida en una función que nos permitirá obtener la temperatura pronosticada

In [ ]:
def solve(inarr, dt, u, v, nt, tstart="2020-05-20T00:00", plot=False, dynamic=False):
    """
    Resuelve la ecuacion de momento en dos dimensiones con velocidades constantes
    
    Parametros
    ----------
    inarr: DataArray
        Objeto de xarray con grilla regular (lat, lon) que
        representa el campo inicial
    dt: int
        Salto de tiempo
    u: float
        Valor de la velocidad zonal
    v: float
        Valor de la velocidad meridional
    nt: int
        Saltos de tiempo a resolver
    tstart: string, opcional
        Fecha inicial de la simulación. Útil para la dimension
        tiempo a agregar en el array de salida. 
        Por defecto toma el valor 2020-05-20T00:00
    
    Retorna
    -------
    outarr: DataArray
        Arreglo con las mismas coordenadas que inarr pero con
        dimension `time` agregada de longitud `nt`+1 y
        saltos `dt`.
    """
    tdelta = pd.Timedelta(dt, unit="s")
    tstart = pd.to_datetime(tstart)
    outarr = xr.DataArray(
        np.nan,
        coords=[
            ("time", pd.date_range(tstart, tstart + (tdelta * nt), freq=f"{dt}s")),
            ("lat", inarr.lat),
            ("lon", inarr.lon),
        ],
    )
    outarr[0] = inarr
    for i in range(outarr.time.size - 1):
        current_time = outarr.isel(time=i)
        cdiffx = current_time.differentiate("lon") / 1000
        cdiffy = current_time.differentiate("lat") / 1000
        outarr[i + 1] = (current_time) - (u * dt * cdiffx) - (v * dt * cdiffy)

    # Colocamos la metadata
    outarr.lat.attrs = inarr.lat.attrs
    outarr.lon.attrs = inarr.lon.attrs
    outarr.attrs = inarr.attrs
    outarr.name = "forecast_temp"

    # En caso de querer retornar el gráfico
    if plot:
        hmap = hv.Dataset(outarr).to(hv.Image, kdims=["lon", "lat"])
        cont = hv.operation.contours(
            hmap, filled=True, dynamic=dynamic, levels=np.arange(-5, 60, 1.0).tolist()
        ).redim.range(forecast_temp=(20, 40))
        xx, yy = np.meshgrid(
            np.linspace(lon.min(), lon.max(), 30), np.linspace(lat.min(), lat.max(), 30)
        )
        vector = hv.VectorField(
            (xx, yy, np.full_like(xx, np.arctan2(v, u)), np.full_like(xx, 1),)
        )
        layout = cont * vector
        layout.opts(
            hv.opts.Polygons(
                tools=["hover", "save"],
                default_tools=[],
                ylim=(lat.min(), lat.max()),
                xlim=(lon.min(), lon.max()),
                width=700,
                height=500,
                colorbar=True,
                color_levels=22,
            ),
            hv.opts.Overlay(default_tools=[]),
            hv.opts.VectorField(default_tools=[]),
        )
        if dynamic:
            return pn.panel(layout, widget_type="scrubber", widget_location="bottom")
        return layout

    return outarr

In [ ]:
fore_temp = solve(temp, 100, u, v, 36)
fore_temp

Exploramos el resultado

In [ ]:
fcst = solve(temp, 100, u, v, 36, plot=True)
fcst

In [ ]:
# Solo guardar cuando se corre localmente
# hv.save(fcst,'resultado',fmt='gif')

---

## Live Demo

En caso de estar visualizando este notebook en binder, podemos modificar algunos parámetros y observar como el campo evoluciona en el tiempo.

In [ ]:
live_panel = interact(
    solve,
    inarr=fixed(temp),
    dt=(10, 110, 10, 100),
    u=(-15, 15, 0.1, u),
    v=(-15, 15, 0.1, v),
    nt=(18, 36 * 5, 18, 36),
    plot=fixed(True),
    dynamic=fixed(True),
)

# Modificamos la velocidad de reproducción por defecto
player = live_panel[1][0][1][1][0]
player.param.interval.default = 200

pn.Row(
    pn.layout.HSpacer(),
    pn.Column(
        pn.pane.PNG(
            "https://repositorio.igp.gob.pe/themes/Mirage/images/logo-igp-corto.png",
            align="center",
            width=250,
            margin=-25,
        ),
        pn.pane.Markdown(
            "## Curso Modelado Numérico de la Atmósfera \n **Parámetros**", margin=0
        ),
        live_panel[0],
        pn.layout.VSpacer(),
        pn.pane.Markdown("*por Gerardo Rivera Tello*"),
        margin=20,
    ),
    live_panel[1],
    pn.layout.HSpacer(),
).servable()